In [1]:
import pandas as pd
import numpy as np

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
drivers = pd.read_csv('drivers.csv', header=0)
constructors = pd.read_csv('constructors.csv', header=0)
results = pd.read_csv('results.csv', header=0)
races = pd.read_csv('races.csv', header=0)

In [3]:
len(races)

1079

In [4]:
# Filer raceId here

#season = 2021 #Enter a F1 World Championship year
#results = results[
#    results['raceId'].isin(
#        races[races['year'] == season]['raceId'].values.tolist()
#        )
#    ]


### Edges

In [5]:
drivers_results = pd.merge(drivers, results, on='driverId')
constructors_results = pd.merge(constructors, results, on='constructorId')

In [6]:
drivers_constructors = pd.merge(drivers_results, constructors_results, on='resultId')
drivers_constructors = drivers_constructors[['driverRef', 'forename', 'surname', 'nationality_x', 'constructorRef', 'name', 'raceId_x']].drop_duplicates()
drivers_constructors.reset_index(inplace=True, drop=True)

In [7]:
dc_1 = pd.merge(drivers_constructors, drivers_constructors, on=['raceId_x', 'constructorRef'])
dc_1 = dc_1[['driverRef_x', 'driverRef_y']]
dc_1['connection'] = 'constructor'
dc_1.drop_duplicates(inplace=True)
#dc_1

In [8]:
dc_2 = pd.merge(drivers_constructors, drivers_constructors, on=['raceId_x', 'nationality_x'])
dc_2 = dc_2[['driverRef_x', 'driverRef_y']]
dc_2['connection'] = 'nation'
dc_2.drop_duplicates(inplace=True)
#dc_2

In [9]:
driver_edges = pd.concat([dc_1,dc_2])

In [10]:
edges = driver_edges[driver_edges['driverRef_x'] != driver_edges['driverRef_y']]
edges.reset_index(inplace=True, drop=True)

In [11]:
edges_list = edges.values.tolist()
edges_set = [set(dl) for dl in edges_list] 
repeat_connections = [idx for idx, ds in enumerate(edges_set) if ds in edges_set[:idx]]

In [12]:
edges.drop(repeat_connections, axis=0, inplace=True)
edges.reset_index(inplace=True, drop=True)

C:\Users\varma\AppData\Local\Temp\ipykernel_32340\1200515550.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  edges.drop(repeat_connections, axis=0, inplace=True)


### Nodes

In [13]:
nodes = drivers[['driverRef', 'forename', 'surname', 'dob', 'nationality']]
nodes.drop_duplicates(inplace=True)
nodes.reset_index(drop=True, inplace=True)

C:\Users\varma\AppData\Local\Temp\ipykernel_32340\3447242104.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nodes.drop_duplicates(inplace=True)


### Writing to files

In [14]:
edges.to_csv('./f1_edges.csv', index = False)
nodes.to_csv('./f1_nodes.csv', index = False)

In [16]:
nodes
edges

,driverRef,forename,surname,dob,nationality
0,hamilton,Lewis,Hamilton,1985-01-07,British
1,heidfeld,Nick,Heidfeld,1977-05-10,German
2,rosberg,Nico,Rosberg,1985-06-27,German
3,alonso,Fernando,Alonso,1981-07-29,Spanish
4,kovalainen,Heikki,Kovalainen,1981-10-19,Finnish
...,...,...,...,...,...
849,aitken,Jack,Aitken,1995-09-23,British
850,tsunoda,Yuki,Tsunoda,2000-05-11,Japanese
851,mazepin,Nikita,Mazepin,1999-03-02,Russian
852,mick_schumacher,Mick,Schumacher,1999-03-22,German


,driverRef_x,driverRef_y,connection
0,hamilton,kovalainen,constructor
1,hamilton,alonso,constructor
2,hamilton,button,constructor
3,hamilton,rosberg,constructor
4,hamilton,bottas,constructor
...,...,...,...
11195,hulkenberg,mick_schumacher,nation
11196,perez,gutierrez,nation
11197,stroll,latifi,nation
11198,norris,aitken,nation
